## Convolutional Neural Networks

Again, I am not going to even try to do a better job than this post...:

https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets/

Now, let's review some of the famous CNN architectures on starting on p.367 of Hands on Machine Learning.

## Python Conv Net

In [39]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, BatchNormalization, BatchNormalization, Dropout
from keras.datasets import mnist
from sklearn.metrics import confusion_matrix
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from skimage.transform import rescale, resize
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

def scale_image(images):
    images = images.reshape(images.shape[0], 28, 28, 1)
    images = images.astype('float32')
    scaled_images = images / 255
    return scaled_images

x_train = scale_image(x_train)
x_test = scale_image(x_test)

In [3]:
x_train.shape

(60000, 28, 28, 1)

In [4]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(5, 5), activation='elu', input_shape=(28, 28, 1), kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, kernel_size=(5, 5), activation='elu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256, activation='elu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='elu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
__________

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy')

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 67s - loss: 0.6560 - val_loss: 0.1152
Epoch 2/5
48000/48000 [==============================] - 71s - loss: 0.2221 - val_loss: 0.0751
Epoch 3/5
33312/48000 [===================>..........] - ETA: 21s - loss: 0.1730

In [7]:
test_predictions = np.argmax(model.predict(x_test),1)

In [8]:
confusion_matrix(y_test, test_predictions)

array([[  0,   0,   0,   0, 801,   0,   0, 144,  31,   4],
       [153, 267,   0,   0, 624,   0,   0,  75,  15,   1],
       [  1,  70,   0,   0, 534,   0,   0, 383,  44,   0],
       [  1, 227,   0,   0, 375,   0,   0, 396,  11,   0],
       [  1,  77,   0,   0, 533,   0,   0, 334,  28,   9],
       [  0,  47,   0,   0, 605,   0,   0, 187,  50,   3],
       [  0,  11,   0,   0, 807,   0,   0, 132,   3,   5],
       [  1, 118,   0,   0, 448,   0,   0, 430,  19,  12],
       [  0, 220,   0,   0, 442,   0,   0, 220,  92,   0],
       [  0, 150,   0,   0, 280,   0,   0, 570,   9,   0]])

In [25]:
np.sum(y_test == test_predictions) / test_predictions.shape

array([ 0.9867])

## Pre-training and Data Augmentation

Source: https://gist.github.com/fchollet/7eb39b44eb9e16e59632d25fb3119975

In [43]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(48,48,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(10, activation='sigmoid'))


# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
top_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Model loaded.


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [30]:
def preprocess_images(images, new_size=(48,48)):
    resized_images = np.array([resize(x, new_size, mode='reflect') for x in images])
    rows, width, height, _ = resized_images.shape
    three_channels = np.zeros((rows, width, height, 3))
    three_channels[:,:,:,0] = resized_images[:,:,:,0]
    three_channels[:,:,:,1] = resized_images[:,:,:,0]
    three_channels[:,:,:,2] = resized_images[:,:,:,0]
    return three_channels

In [31]:
x_train_processed = preprocess_images(x_train)
x_test_processed = preprocess_images(x_test)

In [41]:
datagen = ImageDataGenerator(rotation_range=90, width_shift_range=0.2, height_shift_range=0.2)
generator = datagen.flow(x_train_processed, y_train, batch_size=32)

In [44]:
bottleneck_features_train = base_model.predict_generator(generator, x_train_processed.shape[0] // 32)

KeyboardInterrupt: 

In [33]:
model.fit(x_train_processed, y_train, epochs=1, batch_size=32, validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/1
  576/48000 [..............................] - ETA: 3224s - loss: 9.6906 - acc: 0.1215

KeyboardInterrupt: 

In [13]:
x_train.shape

(60000, 28, 28, 1)